# Imports

In [4]:
import boto3
import json
import dotenv
import os

# Create a session

Remember to active the secret keys in "Add-ons"

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [2]:
region = input("Type in your region: ")

Type in your region:  eu-west-1


In [7]:
try:
    session = boto3.Session(
        aws_access_key_id=user_secrets.get_secret("access_key"),
        aws_secret_access_key=user_secrets.get_secret("secret_access_key"),
        region_name=region
    )
except Exception as e:
    print("Error occurred while creating session: ", e)

# Upload CVSs to S3 Bucket

In [8]:
bucket_name = input('Type in bucket name: ')

Type in bucket name:  fast-bachelor-project-personalize


In [1]:
try:
    session.resource('s3').Bucket(bucket_name).Object('items.csv').upload_file('/kaggle/input/aws-dataset/items.csv') # Change upload_file to your own data
    session.resource('s3').Bucket(bucket_name).Object('users.csv').upload_file('/kaggle/input/aws-dataset/users.csv') # Change upload_file to your own data
    session.resource('s3').Bucket(bucket_name).Object('interactions.csv').upload_file('/kaggle/input/aws-dataset/interactions.csv') # Change upload_file to your own data
except Exception as e:
    print(f"An error occurred while uploading files to S3: {e}")

An error occurred while uploading files to S3: name 'session' is not defined


## Set the S3 bucket policy

In [9]:
s3 = session.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:PutObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

In [10]:
try:
    s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))
    print("Bucket policy has been updated.")
except Exception as e:
    print("An error occurred while updating bucket policy: {}".format(e))

Bucket policy has been updated.
